# Dependencies 

In [0]:
!pip install flask_socketio
!pip install flask_ngrok
! apt install tesseract-ocr
! apt install libtesseract-dev
!pip install tesseract
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [0]:
from fastai.vision import *
from fastai.metrics import error_rate
import numpy as np
from flask_ngrok import run_with_ngrok
from flask import Flask
import os
import logging
from flask import Flask, render_template
from flask_socketio import SocketIO
from PIL import Image 
import cv2
import fastai
from google.colab.patches import cv2_imshow
import numpy as np
from PIL import Image
import string 
import random 
import fastai
import time
%matplotlib inline
from IPython.display import display
import glob
import re
import string
import pytesseract
from PIL import Image
from werkzeug.utils import secure_filename
from flask import Flask, flash, request, redirect, url_for
import shutil 

**The path for Plate detection model is given in the cell below. The export.pkl file is loaded as learner l**

In [0]:
l = load_learner('/content/drive/My Drive/plates model')

# APP


In [0]:
!rm -rf temp
!mkdir temp
!rm -rf '/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages'
!mkdir '/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages'
app = Flask(__name__, template_folder='/content/drive/My Drive/DataForPlateNumFinder2/templates',static_folder='/content/drive/My Drive/DataForPlateNumFinder2/static')
app.secret_key = "abc"
PLATE_FOLDER = os.path.join('static', 'foundplateimages')  
run_with_ngrok(app)   #starts ngrok when the app is run
app.config["ALLOWED_IMAGE_EXTENSIONS"] = ["JPEG", "JPG", "PNG"]
app.config['UP_FOLDER'] = PLATE_FOLDER
print('-------------- App is running--------------')
@app.route('/')
### mainpage() loads the homepage of our application
def mainpage():
    return render_template("Main_Page.html")
### upload renders the page where user uploads the image
@app.route('/predict', methods = ['POST'])  
def upload():  
    return render_template("file_upload_form.html")     
### this function runs when user clicks on try again button. It deletes old predictions adn redirects user to upload page again
@app.route('/tryagain', methods = ['POST'])
def tryagain():
    !rm -rf temp
    !mkdir temp
    !rm -rf '/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages'
    !mkdir '/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages'
    return render_template('file_upload_form.html')

### redirects with clicks on navigation bar 
@app.route('/director', methods = ['POST'])  
def predict():
    if request.form['b1'] == 'Home':
      return render_template('Main_Page.html')
    elif request.form['b1'] == 'About':
      return render_template('about.html')
    else:
      return render_template('file_upload_form.html')

### this function runs after submission of an image 
@app.route('/success', methods = ['POST'])  
def success():  
    if request.method == 'POST':
        start = time.time()    ### notes starting time to find the time taken for prediction   
        ##### snippet below loads the image and crops the borders
        f = request.files['file']
        naym = f.filename 
        f.save(f.filename) 
        im1 = Image.open(f'/content/{naym}')
        width, height = im1.size
        left = int(width * 0.15)
        top = int(height * 0.2)
        right = int(width * 0.85)
        bottom =int(height * 0.8)
        im = im1.crop((left, top, right, bottom))
        ###

        ##### resizes the image for faster execution. Aspect ratio is maintained
        aw, ah = im.size
        if (ah < 300 or aw <300):
          pass
        else:
          ratio = aw/ah
          im = im.resize((int(300*ratio),int(300)), Image.ANTIALIAS)
        ###

        ##### lists and dict to store predictions
        preds=[] 
        predsfull=[]
        dict = {}
        ###

        ##### denoising is done if resolution is low. Also reduces window step size for low quality images
        image = np.array(im, dtype=np.uint8)
        height, width = image.shape[:2]
        if(height<300 and width <300):
          image = cv2.fastNlMeansDenoisingColored(image,None,18,18,7,21)
          step = 6
          print('Denoising was done and step for window was ',step)
        else:
          step =15
          print('Denoising was NOT done and step for window was ',step)
        print('With h,w = ',height,width)
        ###

        ##### implements a siliding windows algorithm to find he number plate and then read characters 
        for (h,w) in [(height//3,width),(height//4,width//2),(height//6,width//3)]: # total 3 windows for prediction
          if(w == width):
            for s in range(0, height-h,step):
              start_row, start_col = int(s), 0
              end_row, end_col = int(s+h), int(w)
              cropped_top = image[start_row:end_row , start_col:end_col]
              img = Image.fromarray(cropped_top, 'RGB')
              fastimg = fastai.vision.image.Image(pil2tensor(img, dtype=np.float32).div_(255))
              pred_class,pred_idx,outputs = l.predict(fastimg) # learner predicts whether there is a plate in the current window
              res =str( pred_class)
              if( res == 'plates'):
                name = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 10)) 
                name = '/content/temp/'+name+'.png'
                # if the plate is found, then 
                text = pytesseract.image_to_string(img) # predicts the characters
                result0 = ''.join(e for e in text if e.isalnum()) #filters results 
                res = ''.join(x for x in result0 if not x.islower()) #filters results 
                if (len(res) >= 5 and len(res) <=10 and res[-1].isdigit() and res[0].isalpha()):
                      predsfull.append(res)  # saves predictions 
                      dict[f'{res}'] = f'{name}'
                      fastimg.save(name)
          else:
              for s in range(0, height-h,step):
                for i in range(0,width-w,step):
                  start_row, start_col = int(s), int(i)
                  end_row, end_col = int(s+h), int(i+w)
                  cropped_top = image[start_row:end_row , start_col:end_col]
                  img = Image.fromarray(cropped_top, 'RGB')
                  fastimg = fastai.vision.image.Image(pil2tensor(img, dtype=np.float32).div_(255))
                  pred_class,pred_idx,outputs = l.predict(fastimg)
                  res =str( pred_class)
                  if( res == 'plates'):
                    name = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 10)) 
                    name = '/content/temp/'+name+'.png'
                    text = pytesseract.image_to_string(img)
                    result0 = ''.join(e for e in text if e.isalnum())
                    res = ''.join(x for x in result0 if not x.islower())
                    if (len(res) >= 8 and len(res) <= 10 and res[-1].isdigit() and res[0].isalpha()):
                          preds.append(res)
                          dict[f'{res}'] = f'{name}' 
                          fastimg.save(name)
        ##### combines results from all three windows 
        pf0 = list(set(sorted(predsfull,key=predsfull.count,reverse=True)))
        p0=list(set(sorted(preds,key=preds.count,reverse=True)))
        new = pf0+ p0
        print('List of predictions by smaller windows -',p0)
        print('List of predictions by bigger window -',pf0)
        print('Final list of predicted number plates - ',new)
        print("Seconds taken to predict - ", (time.time()-start))
        ###

        ##### depending upon the number of different results by the model and tesseract 
        if(len(new)==1):
          content = new[0]
          name = (dict[new[0]])[-14:]
          shutil.copyfile(dict[new[0]], f'/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages/{name}')
          full_filename = os.path.join(app.config['UP_FOLDER'], name)
          return render_template("success_1.html",plateimage = full_filename, text = content)
        elif(len(new)==2):
          content = new[0]
          name = (dict[new[0]])[-14:]
          shutil.copyfile(dict[new[0]], f'/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages/{name}')
          full_filename = os.path.join(app.config['UP_FOLDER'], name)
          content2 = new[1]
          name2 = (dict[new[1]])[-14:]
          shutil.copyfile(dict[new[1]], f'/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages/{name2}')
          full_filename2 = os.path.join(app.config['UP_FOLDER'], name2)
          return render_template("success_2.html", text = content, text2 = content2,plateimage = full_filename,plateimage2 = full_filename2)
        elif(len(new)>=3):
          content =new[0]
          name = (dict[new[0]])[-14:]
          shutil.copyfile(dict[new[0]], f'/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages/{name}')
          full_filename = os.path.join(app.config['UP_FOLDER'], name)
          content2 =new[1]
          name2 = (dict[new[1]])[-14:]
          shutil.copyfile(dict[new[1]], f'/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages/{name2}')
          full_filename2 = os.path.join(app.config['UP_FOLDER'], name2)
          content3 = new[2]
          name3 = (dict[new[2]])[-14:]
          shutil.copyfile(dict[new[2]], f'/content/drive/My Drive/DataForPlateNumFinder2/static/foundplateimages/{name3}')
          full_filename3 = os.path.join(app.config['UP_FOLDER'], name3)
          return render_template("success_3.html",text = content, text2 = content2,text3 = content3,plateimage = full_filename,plateimage2 = full_filename2,plateimage3 = full_filename3)
        else:
          content = 'No good prediction found'
          return render_template("success_0.html", text = content) # if no good result is found 
        ###

app.run()

-------------- App is running--------------
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.6/dist-packages/flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "/usr/local/lib/python3.6/dist-packages/flask_ngrok.py", line 38, in _run_ngrok
    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
IndexError: list index out of range

